# S-parameter Correction and Manipulation
Examples of how to apply a 12-term and 8-term correction model to s-parameter data. In addition the transformation between S and T matrices. 

In [1]:
from pyMeasure import *
import os
import numpy as np

Importing Code.Utils.Names
Importing Code.DataHandlers.NISTModels
Importing Code.DataHandlers.GeneralModels
Importing Code.DataHandlers.TouchstoneModels
Importing Code.DataHandlers.XMLModels
Importing Code.DataHandlers.RadiCALModels
Importing Code.DataHandlers.ZipModels
Importing Code.DataHandlers.Translations
Importing Code.DataHandlers.StatistiCALModels
Importing Code.DataHandlers.MUFModels
Importing Code.Analysis.SParameter
Importing Code.InstrumentControl.Instruments
Importing Code.InstrumentControl.Experiments


In [2]:
# import an s-parameter file to play with
STATISTICAL_TEST_DIRECTORY=r'C:\Users\sandersa\PyCharm Projects\Jupyter-Notebooks\StatistiCAL_Files\StatistiCAL_Tests'
os.chdir(STATISTICAL_TEST_DIRECTORY)
two_port=S2PV1("DY202.txt")
two_port.show()

In [3]:
file_path=r'C:\Users\sandersa\PyCharm Projects\Jupyter-Notebooks\StatistiCAL_Files\StatistiCAL_Tests\CalCoefficients.txt'
in_file=open(file_path,'r')
lines=[]
for line in in_file:
    lines.append(map(lambda x:float(x),line.split(" ")))
in_file.close()

TWELVE_TERM_ERROR_COLUMN_NAMES=["Frequency","reEdf","imEdf","reEsf","imEsf",
                                "reErf","imErf","reExf","imExf","reElf","imElf","reEtf","imEtf",
                                "reEdr","imEdr","reEsr","imEsr","reErr","imErr","reExr","imExr",
                                "reElr","imElr","reEtr","imEtr"]
#"row_pattern":make_row_match_string(TWELVE_TERM_ERROR_COLUMN_NAMES),
options={"data_delimiter":" ","row_end_token":'\n'}

cal_coefficients=lines
complex_cal_coefficients=[]
for row in cal_coefficients:
    frequency=[row[0]]
    complex_numbers=row[1:]
    #print np.array(complex_numbers[1::2])
    complex_array=np.array(complex_numbers[0::2])+1.j*np.array(complex_numbers[1::2])
    complex_cal_coefficients.append(frequency+complex_array.tolist())

In [4]:
corrected_s=correct_sparameters(two_port.sparameter_complex,complex_cal_coefficients)

Could not correct sparameters


TypeError: expected string or buffer

In [5]:
correct_two_port=S2PV1(**{"sparameter_complex":corrected_s,"option_line":"# GHz S RI R 50"})

NameError: name 'corrected_s' is not defined

In [6]:
def compare_s2p_plots(list_S2PV1,**options):
    """compare_s2p_plot compares a list of s2p files ploting each on the same axis"""
    defaults={"format":"MA",
              "display_legend":True,
              "save_plot":False,
              "directory":None,
              "specific_descriptor":"Comparision_Plot",
              "general_descriptor":"Plot",
              "file_name":None,
              "labels":None}
    comparision_plot_options={}
    for key,value in defaults.iteritems():
        comparision_plot_options[key]=value
    for key,value in options.iteritems():
        comparision_plot_options[key]=value
    
    # create a set of 6 subplots    
    fig, compare_axes = plt.subplots(nrows=4, ncols=2, figsize=(8,6),dpi=80)
    if comparision_plot_options["labels"] is None:
        labels=[s2p.path for s2p in list_S2PV1]
    else:
        labels=comparision_plot_options["labels"]
    for s2p_index,s2p in enumerate(list_S2PV1):
        # start by changing the format of all the s2p
        s2p.change_data_format(comparision_plot_options["format"])
        column_names=s2p.column_names[1:]
        for index, ax in enumerate(compare_axes.flat):
            #ax.xaxis.set_visible(False)
            if re.search('arg',column_names[index]):
                ax.set_ylabel('Phase(Degrees)',color='green')
            elif re.search('mag',column_names[index]):
                ax.set_ylabel(r'|${\Gamma} $|',color='green')
            ax.set_title(column_names[index])
            # initial plot of
            x=s2p.get_column('Frequency')
            y=np.array(s2p.get_column(column_names[index]))
            ax.plot(x,y,label=labels[s2p_index])
            if comparision_plot_options["display_legend"]:
                ax.legend(loc=1,fontsize='8')
            
    compare_axes.flat[-2].set_xlabel('Frequency(GHz)',color='k')
    compare_axes.flat[-1].set_xlabel('Frequency(GHz)',color='k')
    fig.subplots_adjust(hspace=0)
    plt.tight_layout()
    # Dealing with the save option
    if comparision_plot_options["file_name"] is None:
        file_name=auto_name(specific_descriptor=comparision_plot_options["specific_descriptor"],
                            general_descriptor=comparision_plot_options["general_descriptor"],
                            directory=comparision_plot_options["directory"]
                            ,extension='png',padding=3)
    else:
        file_name=comparision_plot_options["file_name"]
    if comparision_plot_options["save_plot"]:
        #print file_name
        plt.savefig(os.path.join(comparision_plot_options["directory"],file_name))
    else:
        plt.show()
        

In [ ]:
compare_s2p_plots([two_port,correct_two_port])

In [ ]:
compare_s2p_plots([two_port,correct_two_port],**{"format":"RI","labels":["uncorrected","corrected"]})

In [ ]:
compare_s2p_plots([two_port,correct_two_port],format="MA",labels=["uncorrected","corrected"])

In [ ]:
compare_s2p_plots([two_port,correct_two_port],**{"format":"DB"})

<img src="./Calibration_Example_Files/Twelve_Term_Correction.png" />
VNA Calibration Coefficients

StatistiCAL stores VNA calibration coefficients in the following order:

frequency  Edf  Esf  Erf  Exf  Elf  Etf  Edr  Esr  Err  Exr  Elr  Etr.

The value of the frequency is in gigahertz. The other values are expressed as real/imaginary pairs.


